In [ ]:
#default_exp config

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
#export
import torch
import datetime
import warnings


In [ ]:
#hide
from fastcore.test import test_fail


# Config

Here we define a class `Config` to hold hyperparameters and global variables.

Design from https://github.com/cswinter/DeepCodeCraft/blob/master/hyper_params.py

In [ ]:
#export
class Config: 
    def __init__(self): 
        """Set up default parameters"""
          
        ### Models and datasets
        # options for the pp_model 
        # 1. tuner007/pegasus_paraphrase
        # 2. tdopierre/ProtAugment-ParaphraseGenerator
        # 3. eugenesiow/bart-paraphrase
        self.pp_name = "tuner007/pegasus_paraphrase"
        self.vm_name = "textattack/distilbert-base-uncased-rotten-tomatoes"
        self.sts_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
        self.dataset_name = "rotten_tomatoes"
        
        ### Training hyperparameters
        self.seed = 420
        self.use_fp16 = False
        self.lr = 1e-5   
        self.normalise_rewards = False
        self.pin_memory = True
        self.zero_grad_with_none = False
        self.pad_token_embeddings = False
        self.embedding_padding_multiple = 8
        self.orig_padding_multiple = 8   # pad input to multiple of this
        self.bucket_by_length = True
        self.shuffle_train = False
        self.remove_misclassified_examples = True
        self.unfreeze_last_n_layers = "all"  #counting from the back. set to "all" to do no layer freezing, else set to an int 
        self.reward_fn = "reward_fn_1"
        # This makes the reward function easier to see in wandb
        # copy-paste this from reward function
        self.reward_strategy = "[-0.3 if sts < 0.6   else max(0.2 + v*sts*4, 0) for v,sts in zip(vm_scores, sts_scores)]" 

        ### Paraphrase parameters  
        self.pp = {
            "num_beams": 1, 
            "num_return_sequences": 1, 
            "num_beam_groups": 1, 
            "diversity_penalty": 0.,   # must be a float
            "temperature": 1,
            "length_penalty" : 1,
            "min_length" : 5,
        }
        
        ### Used for testing
        self.use_small_ds = False
        self.n_shards = None
        self.shard_contiguous = None
        
        ### Logging parameters
        self.save_model_while_training = False
        self.save_model_freq = 10
        
        ### These parameters don't do anything yet
        self.sampling_strategy = "greedy"  # doesn't do anything
     
        ### W&B parameters
        self.wandb = dict(
            project = "travis_attack",
            entity = "uts_nlp",
            mode = "online",  # set to "disabled" to turn off wandb, "online" to enable it
            log_grads = False, 
            log_grads_freq = 1,  # no effect if wandb_log_grads is False
            log_token_entropy = True,
            log_token_probabilities = True, 
            run_notes = f"Reward: {self.reward_strategy}\nDataset: {self.dataset_name}"
        )
        
        ### Devices and GPU settings
        #### TODO: do you need this with accelerator? does this handle the post-processing analytics too?
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
        self.devicenum = torch.cuda.current_device() if self.device.type == 'cuda' else -1
        # When not using Accelerator
        #n_wkrs = 4 * torch.cuda.device_count()
        # When using Accelerator 
        self.n_wkrs = 0 
        
        ## Globals 
        self.splits = ['train', 'valid', 'test']
        self.metrics = ['loss', 'pp_logp', 'reward', 'vm_score', "sts_score", 'label_flip']
        self.path_data = "./data/"
        self.path_checkpoints = "../model_checkpoints/travis_attack/"
        self.path_run = None  # keep as None; this is automatically filled out by Trainer class
        self.path_data_cache = "/data/tproth/.cache/huggingface/datasets/"
        self.path_logs = f"./logs/"
        self.path_logfile = self.path_logs + f"run_{datetime.datetime.now().strftime('%Y-%m-%d_%H:%M:%S')}.txt"
        
        # Adjust config depending on dataset. 
        if self.dataset_name   == "simple":           self.adjust_config_for_simple_dataset()
        elif self.dataset_name == "rotten_tomatoes":  self.adjust_config_for_rotten_tomatoes_dataset()  
        
        # Checks
        self._validate_n_epochs()
        
    def adjust_config_for_simple_dataset(self): 
        """Adjust config for the simple dataset."""
        self.dataset_name = "simple"
        self.orig_cname = "text"
        self.label_cname = 'label'
        self.orig_max_length = 20
        self.pp['max_length'] = 20 
        self.batch_size_train = 4
        self.batch_size_eval = 4
        self.acc_steps = 2  
        self.n_train_epochs = 300
        self.eval_freq = 1
        return self
    
    def adjust_config_for_rotten_tomatoes_dataset(self): 
        """Adjust config for the rotten_tomatoes dataset."""
        self.dataset_name = "rotten_tomatoes"
        self.orig_cname = "text"
        self.label_cname = 'label'
        self.orig_max_length = 60  # seems to be the longest that pegasus is trained on. 
        self.pp['max_length'] = 60 
        self.batch_size_train = 16
        self.batch_size_eval = 64
        self.acc_steps = 4
        self.n_train_epochs = 2
        self.eval_freq = 1
        return self    
        
    def small_ds(self):
        """Adjust the config to use a small dataset (for testing purposes).
        Not possible when using the simple dataset. """
        if self.dataset_name == "simple": 
            raise Exception("Don't shard when using the simple dataset (no need)")
        self.use_small_ds = True  # for testing purposes 
        self.n_shards = 300
        self.shard_contiguous = False
        return self
    
    def _validate_n_epochs(self): 
        if self.n_train_epochs % self.eval_freq != 0: 
            raise Exception("Set n_train_epochs to a multiple of eval_freq so there are no leftover epochs.")


In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted index.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.


## Usage

### Basics 

The easiest way is to edit the variables in the config object as you please and then initialise the config object. This will first initialise a set of default values as specified in `__init__()`. Next it calls the methods `adjust_config_for_simple_dataset()` or `adjust_config_for_rotten_tomatoes_dataset()` to overwrite some of these defaults with dataset-specific variables. 

Once ready, call `cfg = Config()` and access values as attributes of `cfg`. For example:

In [ ]:
cfg = Config()
print("Dataset name: ", cfg.dataset_name)
print("Number of train epochs: ", cfg.n_train_epochs)
print("Batch size for train?: ", cfg.batch_size_train)
print("Max paraphrase length?: ", cfg.pp['max_length'])

You can also manually specify which dataset to use by calling the `adjust_config_...` functions yourself. This is useful for writing test cases. 

In [ ]:
cfg = Config().adjust_config_for_simple_dataset()
print("Dataset name: ", cfg.dataset_name)
print("Number of train epochs: ", cfg.n_train_epochs)
print("Batch size for train?: ", cfg.batch_size_train)
print("Max paraphrase length?: ", cfg.pp['max_length'])

In [ ]:
cfg = Config().adjust_config_for_rotten_tomatoes_dataset()
print("Dataset name: ", cfg.dataset_name)
print("Number of train epochs: ", cfg.n_train_epochs)
print("Batch size for train?: ", cfg.batch_size_train)
print("Max paraphrase length?: ", cfg.pp['max_length'])

You can use `vars(cfg)` to get all parameters as a dict: 

In [ ]:
from pprint import pprint
pprint(vars(cfg))

### Using a small dataset for testing

If you want to do testing on a small dataset you can chain on `use_small_ds()` to adjust the config accordingly.

In [ ]:
cfg = Config().adjust_config_for_rotten_tomatoes_dataset().small_ds()
print("Dataset name: ", cfg.dataset_name)
print("Number of train epochs: ", cfg.n_train_epochs)
print("Batch size for train?: ", cfg.batch_size_train)
print("Max paraphrase length?: ", cfg.pp['max_length'])
print("Using small dataset?", cfg.use_small_ds)
print("How many shards?", cfg.n_shards)

This functionality is disabled for the simple dataset because we only have 4 data points for each split. 

In [ ]:
test_fail(Config().adjust_config_for_simple_dataset().adjust_config_for_simple_dataset().small_ds)